## SQLITE

In [157]:
import sqlite3
import pandas as pd

In [158]:
conn = sqlite3.connect("Spacex.sqlite")
cur = conn.cursor()

In [159]:
df = pd.read_csv("Spacex.csv")
df.columns

Index(['Date', 'Time (UTC)', 'Booster_Version', 'Launch_Site', 'Payload',
       'PAYLOAD_MASS__KG_', 'Orbit', 'Customer', 'Mission_Outcome',
       'Landing _Outcome'],
      dtype='object')

#### CREATE THE TABLE

In [160]:
cur.execute("drop table if exists SPACEXTBL")
sqlstr = """
create table SPACEXTBL(
id integer not null primary key autoincrement unique,
Date date,
Time datetime,
Booster_Version text,
Launch_Site text,
Payload text,
Payload_Mass_Kg integer,
Orbit text,
Customer text,
Mission_Outcome text,
Landing_Outcome text
)
"""
cur.execute(sqlstr)
conn.commit()

#### POPULATE THE TABLE

In [161]:
for index, row in df.iterrows():
        cur.execute("""
        insert into SPACEXTBL (Date, Time, Booster_Version, Launch_Site, Payload, Payload_Mass_Kg, Orbit, Customer, 
                                Mission_Outcome, Landing_Outcome) 
                                values(?,?,?,?,?,?,?,?,?,?)""", 
                    (row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9],))
conn.commit()

In [162]:
for row in cur.execute("select * from SPACEXTBL limit 3"):
    print(row)

(1, '04-06-2010', '18:45:00', 'F9 v1.0  B0003', 'CCAFS LC-40', 'Dragon Spacecraft Qualification Unit', 0, 'LEO', 'SpaceX', 'Success', 'Failure (parachute)')
(2, '08-12-2010', '15:43:00', 'F9 v1.0  B0004', 'CCAFS LC-40', 'Dragon demo flight C1, two CubeSats, barrel of Brouere cheese', 0, 'LEO (ISS)', 'NASA (COTS) NRO', 'Success', 'Failure (parachute)')
(3, '22-05-2012', '07:44:00', 'F9 v1.0  B0005', 'CCAFS LC-40', 'Dragon demo flight C2', 525, 'LEO (ISS)', 'NASA (COTS)', 'Success', 'No attempt')


### TASK 1: Display the names of the unique launch sites in the space mission

In [163]:
sqlstr = "select distinct Launch_Site from SPACEXTBL"
view = pd.read_sql_query(sqlstr, conn)
view

,Launch_Site
0,CCAFS LC-40
1,VAFB SLC-4E
2,KSC LC-39A
3,CCAFS SLC-40


### TASK 2: Display 5 records where launch sites begin with the string 'CCA'

In [164]:
sqlstr = "select * from SPACEXTBL where Launch_Site like 'CCA%' limit 5"
view = pd.read_sql_query(sqlstr, conn)
view

,id,Date,Time,Booster_Version,Launch_Site,Payload,Payload_Mass_Kg,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,1,04-06-2010,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute)
1,2,08-12-2010,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute)
2,3,22-05-2012,07:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt
3,4,08-10-2012,00:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt
4,5,01-03-2013,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt


### TASK 3: Display the total payload mass carried by boosters launched by NASA (CRS)

In [165]:
sqlstr = """
select Booster_Version, sum(Payload_Mass_Kg) as Tot_Mass from SPACEXTBL
where Customer = 'NASA (CRS)'
group by 1
order by 2 desc
limit 5
"""

view = pd.read_sql_query(sqlstr, conn)
view

,Booster_Version,Tot_Mass
0,F9 B4 B1039.1,3310
1,F9 FT B1021.1,3136
2,F9 B5 B1058.4,2972
3,F9 FT B1035.1,2708
4,F9 B4 B1045.2,2697


### TASK 4: Display average payload mass carried by booster version F9 v1.1

In [166]:
sqlstr = """
select avg(Payload_Mass_Kg) as avg_payload from SPACEXTBL
where Booster_Version like 'F9 v1.1%'
"""

view = pd.read_sql_query(sqlstr, conn)
view

,avg_payload
0,340.4


### TASK 5: List the date when the first successful landing outcome in ground pad was acheived.

In [167]:
sqlstr = """
select min(date) as first_groundpad_landing from SPACEXTBL 
where landing_outcome = 'Success (ground pad)'
"""

view = pd.read_sql_query(sqlstr, conn)
view

,first_groundpad_landing
0,01-05-2017


### TASK 6: List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

In [168]:
sqlstr = """
select distinct Booster_Version from SPACEXTBL 
where landing_outcome = 'Success (drone ship)'
and Payload_Mass_Kg between 4000 and 6000
"""

view = pd.read_sql_query(sqlstr, conn)
view

,Booster_Version
0,F9 FT B1022
1,F9 FT B1026
2,F9 FT B1021.2
3,F9 FT B1031.2


### TASK 7: List the total number of successful and failure mission outcomes

In [169]:
sqlstr = """
select  CASE when Mission_Outcome like 'Failure%'
            then 'Failure'
            else 'Success'
        END as Outcome, 
        count(*) as count
        from SPACEXTBL
group by Outcome
order by 2 desc
"""

view = pd.read_sql_query(sqlstr, conn)
view

,Outcome,count
0,Success,100
1,Failure,1


### TASK 8: List the names of the booster_versions which have carried the maximum payload mass. Use a subquery

In [170]:
sqlstr = """
SELECT BOOSTER_VERSION, PAYLOAD_MASS_KG FROM SPACEXTBL 
WHERE PAYLOAD_MASS_KG = (SELECT MAX(PAYLOAD_MASS_KG) FROM SPACEXTBL)
"""

view = pd.read_sql_query(sqlstr, conn)
view

,Booster_Version,Payload_Mass_Kg
0,F9 B5 B1048.4,15600
1,F9 B5 B1049.4,15600
2,F9 B5 B1051.3,15600
3,F9 B5 B1056.4,15600
4,F9 B5 B1048.5,15600
5,F9 B5 B1051.4,15600
6,F9 B5 B1049.5,15600
7,F9 B5 B1060.2,15600
8,F9 B5 B1058.3,15600
9,F9 B5 B1051.6,15600


### TASK 9: List the failed landing_outcomes in drone ship, their booster versions, and launch site names for in year 2015

In [171]:
#Failure (drone ship)
sqlstr = """
SELECT * FROM SPACEXTBL
where Landing_Outcome = 'Failure (drone ship)' and Date like '%2015'
"""

view = pd.read_sql_query(sqlstr, conn)
view

,id,Date,Time,Booster_Version,Launch_Site,Payload,Payload_Mass_Kg,Orbit,Customer,Mission_Outcome,Landing_Outcome
0,14,10-01-2015,09:47:00,F9 v1.1 B1012,CCAFS LC-40,SpaceX CRS-5,2395,LEO (ISS),NASA (CRS),Success,Failure (drone ship)
1,17,14-04-2015,20:10:00,F9 v1.1 B1015,CCAFS LC-40,SpaceX CRS-6,1898,LEO (ISS),NASA (CRS),Success,Failure (drone ship)
